# 单电机实验 demo

## 初始化电机

In [2]:
from robodyno.components.can_bus.mi_motor import MiMotor
from robodyno.interfaces import CanBus
can = CanBus()

motor = MiMotor(can, 0x7F)

## 使能

In [3]:
motor.enable()

## 控制

In [ ]:
import ipywidgets as widgets
from IPython.display import display
import time
import threading
import numpy as np
from collections import deque

thread_running = True

start_time = time.time()  # 获取开始时间

# 数据存储（记录时间、位置、速度、力矩）
MAX_DATA_POINTS = 1000
time_data = deque(maxlen=MAX_DATA_POINTS)
pos_data = deque(maxlen=MAX_DATA_POINTS)
vel_data = deque(maxlen=MAX_DATA_POINTS)
torque_data = deque(maxlen=MAX_DATA_POINTS)

# 电机使能按钮
is_enabled = (motor.get_state()[0].value != 0)
enable_motor_button = widgets.ToggleButton(
    value=is_enabled,
    description='电机已使能' if is_enabled else '电机已失能',
    button_style='success' if is_enabled else 'danger',
    icon='check' if is_enabled else 'times',
    tooltip='启用或禁用电机',
)
def on_toggle(change):
    if change.new:  # 启用电机
        motor.enable()
        enable_motor_button.description = '电机已使能'
        enable_motor_button.button_style = 'success'
        enable_motor_button.icon = 'check'
    else:  # 禁用电机
        motor.disable()
        enable_motor_button.description = '电机已失能'
        enable_motor_button.button_style = 'danger'
        enable_motor_button.icon = 'times'
enable_motor_button.observe(on_toggle, names='value')

# 初始化位置按钮
init_pos_buttom = widgets.Button(description='初始化位置')
def on_init_pos_button_clicked(b):
    motor.init_pos()
init_pos_buttom.on_click(on_init_pos_button_clicked)

# 数据显示控件
pos_label = widgets.Label(value="0.00 rad", description="位置:")
vel_label = widgets.Label(value="0.00 rad/s", description="速度:")
torque_label = widgets.Label(value="0.00 Nm", description="力矩:")
voltage_label = widgets.Label(value="0.00 V", description="电压:")
temp_label = widgets.Label(value="0.00 °C", description="温度:")

# 数据显示区域布局
data_display_area = widgets.VBox([
    widgets.HTML("<b>实时数据</b>"),  # 标题
    widgets.HBox([widgets.Label("位置:"), pos_label], layout=widgets.Layout(margin="5px")),
    widgets.HBox([widgets.Label("速度:"), vel_label], layout=widgets.Layout(margin="5px")),
    widgets.HBox([widgets.Label("力矩:"), torque_label], layout=widgets.Layout(margin="5px")),
    widgets.HBox([widgets.Label("电压:"), voltage_label], layout=widgets.Layout(margin="5px")),
    widgets.HBox([widgets.Label("温度:"), temp_label], layout=widgets.Layout(margin="5px"))
])

# 控制模式切换按钮
mode = motor.get_mode()[0].value
toggle_buttons = widgets.ToggleButtons(
    options=[('MIT', 0), ('位置', 1), ('速度', 2), ('力矩', 3)],
    description='控制模式:',
    value=mode
)
# 参数输入控件
mit_controls = widgets.VBox([
    widgets.FloatText(description='Torque:', value=0.0),
    widgets.FloatText(description='Position:', value=0.0),
    widgets.FloatText(description='Velocity:', value=0.0),
    widgets.FloatText(description='Kp:', value=0.0),
    widgets.FloatText(description='Kd:', value=0.0),
    widgets.Button(description='设置MIT参数')
])
pos_controls = widgets.VBox([
    widgets.FloatText(description='位置:', value=0.0),
    widgets.FloatText(description='速度限制:', value=30.0),
    widgets.Button(description='设置位置参数')
])
vel_controls = widgets.VBox([
    widgets.FloatText(description='速度:', value=0.0),
    widgets.FloatText(description='电流限制:', value=20.0),
    widgets.Button(description='设置速度参数')
])
torque_controls = widgets.VBox([
    widgets.FloatText(description='力矩:', value=0.0),
    widgets.Button(description='设置力矩参数')
])
def update_controls(mode):
    if mode == 0:  # MIT 模式
        display_area.children = [mit_controls]
    elif mode == 1:  # 位置模式
        display_area.children = [pos_controls]
    elif mode == 2:  # 速度模式
        display_area.children = [vel_controls]
    elif mode == 3:  # 力矩模式
        display_area.children = [torque_controls]
def on_value_change(change):
    mode = change.new
    update_controls(mode)
toggle_buttons.observe(on_value_change, names='value')

# 为每个模式的设置按钮添加功能
def set_mit_params(_):
    motor.mit_mode()
    params = [control.value for control in mit_controls.children[:-1]]
    motor.set_mit(*params)

def set_pos_params(_):
    motor.position_mode()
    pos = pos_controls.children[0].value
    vel_limit = pos_controls.children[1].value
    motor.set_vel_limit(vel_limit)
    motor.set_pos(pos)


def set_vel_params(_):
    motor.velocity_mode()
    velocity = vel_controls.children[0].value
    current_limit = vel_controls.children[1].value
    motor.set_current_limit(current_limit)
    motor.set_vel(velocity)


def set_torque_params(_):
    motor.torque_mode()
    torque = torque_controls.children[0].value
    motor.set_torque(torque)

# 为按钮绑定事件
mit_controls.children[-1].on_click(set_mit_params)
pos_controls.children[-1].on_click(set_pos_params)
vel_controls.children[-1].on_click(set_vel_params)
torque_controls.children[-1].on_click(set_torque_params)

# 控件显示区域
display_area = widgets.VBox()
update_controls(mode)


# 显示所有控件

display(enable_motor_button, init_pos_buttom, data_display_area, toggle_buttons, display_area)



# 数据实时更新函数
def update_data_display():
    while thread_running:
        try:
            # 获取实时数据
            _, _, _, p, v, t, _ = motor._mi_get_state()
            pos = motor.get_pos(timeout=1)
            vel = motor.get_vel(timeout=1)
            torque = motor.get_torque(timeout=1)
            torque = t
            voltage = motor.get_voltage(timeout=1)
            temp = motor.get_temperature(timeout=1)

            # 获取当前时间
            current_time = time.time() - start_time

            if pos is None or vel is None or torque is None or voltage is None or temp is None:
                raise Exception("获取数据失败")
            else:
                # 更新显示控件的值，固定小数位数和宽度
                pos_label.value = f"{pos:+8.2f} rad"        # 正数显示+，负数显示-
                vel_label.value = f"{vel:+8.2f} rad/s"
                torque_label.value = f"{torque:+8.2f} Nm"
                voltage_label.value = f"{voltage:+8.2f} V"
                temp_label.value = f"{temp:+8.2f} °C"
                
                # 更新数据记录
                global time_data, pos_data, vel_data, torque_data
                time_data.append(current_time)
                pos_data.append(pos)
                vel_data.append(vel)
                torque_data.append(torque)

        except Exception as e:
            continue
        
        time.sleep(0.05)  # 更新频率

# 启动数据更新线程
update_thread = threading.Thread(target=update_data_display, daemon=True)
update_thread.start()

## 绘图

In [ ]:
import matplotlib.pyplot as plt
import numpy as np
from matplotlib.animation import FuncAnimation

# 创建 x 轴数据
x = np.linspace(0, 2 * np.pi, 100)

# 初始化图形
fig, ax = plt.subplots()

# 设置图形的初始内容
line_sin, = ax.plot(time_data, pos_data, label="pos", color='blue')
line_cos, = ax.plot(time_data, vel_data, label="vel", color='green')
line_cos, = ax.plot(time_data, torque_data, label="torque", color='red')

# 设置标题和标签
ax.set_title("Real-time Sine and Cosine Functions")
ax.set_xlabel("x")
ax.set_ylabel("y")
ax.legend()

# 动画更新函数
def update(frame):
    # 更新 sin(x) 和 cos(x) 数据
    line_sin.set_ydata(np.sin(x + frame / 10))  # 动态更新 sin(x)
    line_cos.set_ydata(np.cos(x + frame / 10))  # 动态更新 cos(x)
    return line_sin, line_cos

# 创建动画
ani = FuncAnimation(fig, update, frames=100, interval=50)

# 显示图形
plt.show()


## 停止

In [5]:
thread_running = False
update_thread.join()

## 电机失能

In [6]:
motor.disable()

## 断开CAN总线

In [6]:
can.disconnect()